In [1]:
!pip install datasets torch_optimizer lion_pytorch clang_repl_kernel --break-system-packages
!pip install --upgrade clang-repl-kernel  --break-system-packages
#!apt update
#!apt-get install libc++1 libc++abi1

DEPRECATION: Loading egg at /usr/local/lib/python3.12/dist-packages/torchtext-0.18.0a0+9bed85d-py3.12-linux-x86_64.egg is deprecated. pip 25.1 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at /usr/local/lib/python3.12/dist-packages/torchaudio-2.6.0a0+d883142-py3.12-linux-x86_64.egg is deprecated. pip 25.1 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at /usr/local/lib/python3.12/dist-packages/looseversion-1.3.0-py3.12.egg is deprecated. pip 25.1 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at /usr/local/lib/python3.12/dist-packages/opt_einsum-3.4.0-py3.12

In [2]:
from ClangReplInterface import ClangReplInterface
clang_repl = ClangReplInterface()


In [3]:
import os
import copy
import json
import torch
import gc
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    DataCollatorForLanguageModeling,
)
from transformers.optimization import Adafactor
from datasets import Dataset

# For hyperparameter optimization
import optuna
import pickle
from Config import SimpleConfig
from ClangReplInterface import ClangReplInterface

ref_checkpoint_path = "./saved_models/sample/checkpoint.pt"
last_checkpoint_path = "./saved_models/reasoning/checkpoint.pt"
checkpoint_dir_pre = "./saved_models/reasoning/epoch_"

test_target_object_file = "./convert/test_target_and_object/converted_data.json"

config = SimpleConfig()


/usr/local/lib/python3.12/dist-packages/transformers/utils/hub.py:106: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


INFO:datasets:PyTorch version 2.7.0a0+ecf3bae40a.nv25.2 available.
INFO:datasets:Polars version 1.14.0 available.


In [4]:
test=""">>> std::string getGreeting() {return "Hello";}\n>>> template<typename T> T bypass(T value) {return value;}\n>>>  const std::string greeting("hel");\n>>> %<< ( greeting);\nHello"""
clang_repl = ClangReplInterface()
result, response = clang_repl.run_verify(test)
response

'Expected Output: Hello, Actual Output: hel'

In [5]:
def load_sample_dataset(pk_file):
    with open(config.dataset_file, "rb") as f:
        global_samples = pickle.load(f)
        sample_dataset = []
        for sample in global_samples:
            sample_dataset.append({"content": sample + "<|endoftext|>"})
        return sample_dataset

In [6]:
train_data_sample = load_sample_dataset(config.dataset_file)

In [7]:
import re

def remove_comments(code: str):
    pattern = re.compile(r'//.*?$|/\*.*?\*/', re.DOTALL | re.MULTILINE)
    return re.sub(pattern, '', code)


def find_all_tag_indexes(text, tag):
    """Return a list of starting indexes where the tag occurs in the text."""
    indexes = []
    start = 0
    while True:
        idx = text.find(tag, start)
        if idx == -1:
            break
        indexes.append(idx)
        start = idx + len(tag)
    return indexes


def get_tag_start_end(idx, starts, ends, tag, full_text):
    start = starts[idx]+len(tag)
    end = ends[idx]
    return full_text[start: end].strip()
    

def reward_correct(full_text):
    # handle only first answer
    reward = 0.0
    test_target_open = find_all_tag_indexes(full_text, "<Test Target>")
    test_target_close = find_all_tag_indexes(full_text, "</Test Target>")
    clang_repl_open = find_all_tag_indexes(full_text, "<Clang-repl Test>")
    clang_repl_close = find_all_tag_indexes(full_text, "</Clang-repl Test>")
    if len(test_target_open) == 0 or len(test_target_close) == 0 or len(clang_repl_open) == 0 or len(clang_repl_close) == 0:
        return reward, '<Test Target> or <Clang-repl Test> not found'
    if len(test_target_open) != len(test_target_close) or len(clang_repl_open) != len(clang_repl_close):
        return reward , '<Test Target> or <Clang-repl Test> pair not match'
    if not all(x < y for x, y in zip(test_target_open, test_target_close)):
        return reward, '<Test Target> not closed properly'
    if not all(x < y for x, y in zip(clang_repl_open, clang_repl_close)):
        return reward, '<Clang-repl Test> not closed properly' 
    target_text = get_tag_start_end(-1, test_target_open, test_target_close, "<Test Object>", full_text)
    target_text = remove_comments(target_text)
    target_text = ">>> "+target_text.replace('\n', '')

    for idx in range(len(clang_repl_open)):
        clang_repl_test = get_tag_start_end(idx, clang_repl_open, clang_repl_close, "<Clang-repl Test>", full_text)
        clang_repl = ClangReplInterface()
        test_case_with_target = target_text+'\n'+clang_repl_test
        #print(test_case_with_target)
        result, response = clang_repl.run_verify(test_case_with_target)
        reward = 0.0
        if result == 'ok':
            reward = 2.0
        elif result == 'fail':
            reward = 1.0
        elif result == 'error':
            reward = 0.0
        else:
            assert False
        return reward, response
    else:
        return reward, ''

In [8]:
for item in train_data_sample:
    content = item['content']
    reward, response = reward_correct(content)
    if reward != 2.0:
        print(reward)

In [9]:
import re
import json
from collections import Counter

def process_by_chunks(file_content):
    """Process the file content by splitting it into chunks and extracting test cases."""
    chunks = file_content.split('},\n{')
    test_cases = []
    
    for i, chunk in enumerate(chunks):
        # Fix the first and last chunks
        if i == 0:
            chunk = chunk[1:]  # Remove leading [
        if i == len(chunks) - 1:
            chunk = chunk[:-2]  # Remove trailing ]
        
        # Add the curly braces back that were removed in the split
        if i > 0:
            chunk = '{' + chunk
        if i < len(chunks) - 1:
            chunk = chunk + '}'
        
        # Find category
        category_match = re.search(r'"category": "([^"]+)"', chunk)
        if not category_match:
            continue
        category = category_match.group(1)
        
        # Find content
        content_start = chunk.find('"content": "') + 12
        content_end = chunk.rfind('"')
        
        if content_start < 12 or content_end <= content_start:
            continue
        
        content = chunk[content_start:content_end]
        
        # Process content to remove newlines between tags
        content = content.replace('\n', ' ')
        content = re.sub(r'<Test Case>\s+', '<Test Case>', content)
        content = re.sub(r'\s+</Test Case>', '</Test Case>', content)
        content = re.sub(r'<Test Target Object>\s+', '<Test Target Object>', content)
        content = re.sub(r'\s+</Test Target Object>', '</Test Target Object>', content)
        content = re.sub(r'<Test Target>\s+', '<Test Target>', content)
        content = re.sub(r'\s+</Test Target>', '</Test Target>', content)
        
        test_cases.append({
            "category": category,
            "content": content
        })
    
    return test_cases

def run():
    test_target_object_file = "manual_data_set/ReasoningTestTarget.json"
    test_target_object_out_file = "manual_data_set/ReasoningTestTarget_process.json"
    # Read the file content
    with open(test_target_object_file, 'r', encoding='utf-8') as file:
        file_content = file.read()
    
    # Process the file
    test_cases = process_by_chunks(file_content)
    print(f"Processed {len(test_cases)} test cases by chunks")

    with open(test_target_object_out_file, 'w') as json_file:
        json.dump(test_cases, json_file, indent=4)
    
    # Validate that we have proper test cases
    if test_cases:
        print("Sample test case:")
        print(json.dumps(test_cases[0], indent=2))
        
        # Count test cases by category
        category_counts = Counter(tc["category"] for tc in test_cases)
        
        print("Test cases by category:")
        for category, count in category_counts.items():
            print(f"{category}: {count}")
        
        # Print sample from each category
        print("\nSample from each category:")
        seen_categories = set()
        for tc in test_cases:
            if tc["category"] not in seen_categories:
                print(f"Category: {tc['category']}")
                content_preview = tc["content"][:100] + "..." if len(tc["content"]) > 100 else tc["content"]
                print(f"Content preview: {content_preview}")
                print()
                seen_categories.add(tc["category"])
        
        # Save the processed data to a file
        with open('processed_test_cases.json', 'w', encoding='utf-8') as outfile:
            json.dump(test_cases, outfile, indent=2)
        
        print(f"Created JSON with {len(test_cases)} test cases across {len(category_counts)} categories")
        print("Data saved to 'processed_test_cases.json'")
    else:
        print("No test cases were found")

run()

Processed 1 test cases by chunks
Sample test case:
{
  "category": "simple arithmetic",
  "content": "<Test Case><Test Target Object>Add two integers and return the sum.</Test Target Object> <Test Target>int add(int a, int b) {     return a + b; }</Test Target> <Test Case>\"     },     {         \"category\": \"simple arithmetic\",         \"content\": \"<Test Case><Test Target Object>Subtract one integer from another and return the difference.</Test Target Object> <Test Target>int subtract(int a, int b) {     return a - b; }</Test Target> <Test Case>\"     },     {         \"category\": \"simple arithmetic\",         \"content\": \"<Test Case><Test Target Object>Multiply two integers and return the product.</Test Target Object> <Test Target>int multiply(int a, int b) {     return a * b; }</Test Target> <Test Case>\"     },     {         \"category\": \"simple arithmetic\",         \"content\": \"<Test Case><Test Target Object>Divide one integer by another and return the quotient.</Tes

In [10]:

def get_test_target_content(full_text):
    test_target_open = find_all_tag_indexes(full_text, "<Test Target>")
    test_target_close = find_all_tag_indexes(full_text, "</Test Target>")
    target_text = get_tag_start_end(-1, test_target_open, test_target_close, "<Test Target>", full_text)
    return target_text

def load_reasoning_dataset(test_target_object_file):
    with open(test_target_object_file, 'r', encoding='utf-8') as file:
        data = json.load(file)
        train = []
        val = []
        categories = set()
        data_dic = {}
        for item in data:
            categories.add(item['category'])
        for cat in categories:
            data_dic[cat] = []
        for item in data:
            data_dic[item['category']].append(item['content'])
        for cat in categories:
            for idx, item in enumerate(data_dic[cat]):
                content = f"### Instruction\n\nn<Test Target>\n{get_test_target_content(item)}\n</Test Target>\nWrtie a Clang-repl Test\n### Response\n"
                if idx >=14:
                    val.append(({"content":content}))
                else:
                    train.append(({"content":content}))

        return train, val

test_target_object_file = "manual_data_set/ReasoningTestTarget.json"
reasoning_dataset, val_reasoning_dataset = load_reasoning_dataset(test_target_object_file)

In [11]:
reasoning_dataset

[{'content': "### Instruction\n\nn<Test Target>\nvoid toUpperCase(char* str) {     for (int i = 0; str[i] != '\\\\0'; i++) {         if (str[i] >= 'a' && str[i] <= 'z') {             str[i] = str[i] - 32;         }     } }\n</Test Target>\nWrtie a Clang-repl Test\n### Response\n"},
 {'content': "### Instruction\n\nn<Test Target>\nvoid toLowerCase(char* str) {     for (int i = 0; str[i] != '\\\\0'; i++) {         if (str[i] >= 'A' && str[i] <= 'Z') {             str[i] = str[i] + 32;         }     } }\n</Test Target>\nWrtie a Clang-repl Test\n### Response\n"},
 {'content': "### Instruction\n\nn<Test Target>\nvoid reverseString(char* str) {     int length = 0;     while (str[length] != '\\\\0') {         length++;     }          for (int i = 0; i < length / 2; i++) {         char temp = str[i];         str[i] = str[length - 1 - i];         str[length - 1 - i] = temp;     } }\n</Test Target>\nWrtie a Clang-repl Test\n### Response\n"},
 {'content': "### Instruction\n\nn<Test Target>\nint c

In [12]:
from torchtext.data.utils import get_tokenizer

# Instantiate torchtext's basic English tokenizer
tokenizer = get_tokenizer("basic_english")

# Iterate over the dataset, tokenize the content, and print token lengths
for data in reasoning_dataset:
    content = data['content']
    # Tokenize the content using the basic English tokenizer
    tokens = tokenizer(content)
    print(f"Token length: {len(tokens)}\n")

Token length: 57

Token length: 57

Token length: 69

Token length: 112

Token length: 147

Token length: 53

Token length: 69

Token length: 103

Token length: 105

Token length: 108

Token length: 52

Token length: 57

Token length: 53

Token length: 84

Token length: 39

Token length: 44

Token length: 39

Token length: 44

Token length: 42

Token length: 61

Token length: 43

Token length: 39

Token length: 57

Token length: 42

Token length: 95

Token length: 63

Token length: 42

Token length: 41

Token length: 42

Token length: 42

Token length: 39

Token length: 42

Token length: 46

Token length: 55

Token length: 45

Token length: 49

Token length: 54

Token length: 46

Token length: 44

Token length: 52

Token length: 59

Token length: 85

Token length: 33

Token length: 23

Token length: 25

Token length: 32

Token length: 32

Token length: 52

Token length: 55

Token length: 50

Token length: 42

Token length: 45

Token length: 73

Token length: 51

Token length: 47

Token

/usr/local/lib/python3.12/dist-packages/torchtext-0.18.0a0+9bed85d-py3.12-linux-x86_64.egg/torchtext/data/__init__.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is deprecated and the last released version will be 0.18 (this one). You can silence this warning by calling the following at the beginnign of your scripts: `import torchtext; torchtext.disable_torchtext_deprecation_warning()`
  warnings.warn(torchtext._TORCHTEXT_DEPRECATION_MSG)


In [13]:
val_reasoning_dataset

[{'content': "### Instruction\n\nn<Test Target>\nint stringToInt(const char* str) {     int result = 0;     int sign = 1;     int i = 0;          if (str[0] == '-') {         sign = -1;         i = 1;     }          while (str[i] != '\\\\0') {         if (str[i] >= '0' && str[i] <= '9') {             result = result * 10 + (str[i] - '0');         }         i++;     }          return sign * result; }\n</Test Target>\nWrtie a Clang-repl Test\n### Response\n"},
 {'content': '### Instruction\n\nn<Test Target>\nint sumEvenElements(int arr[], int n) {     if (n <= 0) {         return 0;     }          int sum = sumEvenElements(arr, n - 1);     if (arr[n-1] % 2 == 0) {         sum += arr[n-1];     }     return sum; }\n</Test Target>\nWrtie a Clang-repl Test\n### Response\n'},
 {'content': '### Instruction\n\nn<Test Target>\ndouble calculateAverage(int arr[], int size) {     int sum = 0;     for (int i = 0; i < size; i++) {         sum += arr[i];     }     return (double)sum / size; }\n</Test 

In [14]:
def _target_check(target_text):
    clang_repl = ClangReplInterface()
    test_case_with_target = ">>> "+target_text+'\n'
    #print(test_case_with_target)
    result, response = clang_repl.run_verify(test_case_with_target)
    reward = 0.0
    if result == 'ok':
        reward = 2.0
    elif result == 'fail':
        reward = 1.0
    elif result == 'error':
        reward = 0.0
    else:
        assert False
    return reward, response


with open(test_target_object_file, 'r', encoding='utf-8') as file:
    data = json.load(file)
    for item in train_data_sample:
        content = item['content']
        target = get_test_target_content(content)
        target = target.replace('\n', '')
        reward, response = _target_check(target)
        if reward != 2.0:
            print(reward)